In [ ]:
import os
from IPython import get_ipython
get_ipython().run_line_magic("matplotlib", "inline")
from PIL import Image
import numpy as np

def process_images(equalized_dir, masked_dir, output_dir):
    for root, dirs, files in os.walk(equalized_dir):
        for file in files:
            if file.endswith(('.jpg', '.jpeg', '.png')):
                # Get the relative path for maintaining folder structure
                relative_path = os.path.relpath(root, equalized_dir)

                # Paths to the equalized image and corresponding mask
                equalized_path = os.path.join(root, file)
                mask_path = os.path.join(masked_dir, relative_path, file)
                mask_path = mask_path.replace(".jpeg", "_mask.png")  # Adjust for mask naming conventions

                # Ensure the mask exists
                if not os.path.exists(mask_path):
                    print(f"Mask not found for: {equalized_path}")
                    continue

                # Open the equalized image and mask
                equalized_image = Image.open(equalized_path).convert('RGB')
                mask_image = Image.open(mask_path).convert('L')  # Convert mask to grayscale

                # Resize the mask to match the image dimensions
                mask_image = mask_image.resize(equalized_image.size, resample=Image.BILINEAR)

                # Multiply the image and mask
                equalized_array = np.array(equalized_image)
                mask_array = np.array(mask_image) / 255.0  # Normalize mask to [0, 1]
                masked_image = (equalized_array * mask_array[..., None]).astype(np.uint8)

                # Create the output directory structure
                output_folder = os.path.join(output_dir, relative_path)
                os.makedirs(output_folder, exist_ok=True)

                # Save the masked image
                output_path = os.path.join(output_folder, file)
                Image.fromarray(masked_image).save(output_path)

                print(f"Processed and saved: {output_path}")

# Input and output directories
equalized_dir = "../data/equalized"
masked_dir = "../data/masked"
output_dir = "../data/premultiplied"

# Process the images
process_images(equalized_dir, masked_dir, output_dir)
